In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('Data/maintenance.csv')

# Display the first few rows of the DataFrame
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Vibration Levels,Operational Hours,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,42.0,20.0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,52.0,21.0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,44.0,18.0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,52.0,10.0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,44.0,10.0,No Failure


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      500 non-null    int64  
 1   Product ID               500 non-null    object 
 2   Type                     500 non-null    object 
 3   Air temperature [K]      500 non-null    float64
 4   Process temperature [K]  500 non-null    float64
 5   Rotational speed [rpm]   500 non-null    int64  
 6   Torque [Nm]              500 non-null    float64
 7   Vibration Levels         500 non-null    float64
 8   Operational Hours        500 non-null    float64
 9   Failure Type             500 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 39.2+ KB


In [3]:
df.isnull().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Vibration Levels           0
Operational Hours          0
Failure Type               0
dtype: int64

In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df.dtypes

UDI                          int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Vibration Levels           float64
Operational Hours          float64
Failure Type                object
dtype: object

In [15]:
df.columns = df.columns.str.strip()


In [19]:
print(df.columns.tolist())


['UDI', 'Product ID', 'Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Vibration Levels', 'Operational Hours', 'Failure Type']


In [20]:
def generate_inspection_comment(row):
    engine = row['UDI']
    air_temp = row['Air temperature [K]']
    proc_temp = row['Process temperature [K]']
    rpm = row['Rotational speed [rpm]']
    torque = row['Torque [Nm]']
    vibration = row['Vibration Levels']
    hours = row['Operational Hours']
    failure = row['Failure Type']

    comment = (
        f"Technician inspected Engine #{engine}. RPM at {rpm}, torque at {torque:.1f} Nm. "
        f"Air temp {air_temp:.1f}K, Process temp {proc_temp:.1f}K. "
        f"Vibration level recorded at {vibration:.1f}. Engine has run for {hours:.1f} hours. "
    )

    if failure == "No Failure":
        comment += "No failure detected."
    elif "Tool" in failure:
        comment += "Signs of tool wear observed. Recommend inspection."
    else:
        comment += "Anomaly present. Further diagnostics required."

    return comment



In [ ]:
df['Comment'] = df.apply(generate_inspection_comment, axis=1)


In [24]:

df.to_csv('Data/maintenance_comments.csv', index=False)


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Vibration Levels,Operational Hours,Failure Type,Comment
0,1,M14860,M,298.1,308.6,1551,42.8,42.0,20.0,No Failure,"Technician inspected Engine #1. RPM at 1551, t..."
1,2,L47181,L,298.2,308.7,1408,46.3,52.0,21.0,No Failure,"Technician inspected Engine #2. RPM at 1408, t..."
2,3,L47182,L,298.1,308.5,1498,49.4,44.0,18.0,No Failure,"Technician inspected Engine #3. RPM at 1498, t..."
3,4,L47183,L,298.2,308.6,1433,39.5,52.0,10.0,No Failure,"Technician inspected Engine #4. RPM at 1433, t..."
4,5,L47184,L,298.2,308.7,1408,40.0,44.0,10.0,No Failure,"Technician inspected Engine #5. RPM at 1408, t..."


In [ ]:
df.head()

In [31]:
import openai
import gradio as gr


ModuleNotFoundError: No module named 'gradio'

In [32]:
def generate_sap_output(comment):
    prompt = f"""
You are a smart SAP Plant Maintenance assistant. Read this technician inspection comment and convert it into a structured SAP breakdown notification.

Comment: "{comment}"

Return the following fields:
Asset:
Issue:
Priority:
Action:
Notification Type:
Status:
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response['choices'][0]['message']['content']

In [33]:
for i in range(3):
    print(generate_sap_output(df['Comment'].iloc[i]))

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:

# ✅ Your OpenAI API key here
openai.api_key = "sk-xxxxxxx"

# 🔧 The GenAI function
def generate_work_order(comment):
    prompt = f"""You are a maintenance assistant. Convert inspection comments into structured SAP-style work orders.

Examples:

Comment: "Pump 7 leaking from main valve"
→
Asset: Pump 7
Issue: Leak at main valve
Priority: High
Action: Replace valve gasket
Notification Type: M2 (Breakdown)
Status: Open

Comment: "Conveyor belt jammed overnight"
→
Asset: Conveyor
Issue: Jam
Priority: Medium
Action: Inspect rollers, remove obstruction
Notification Type: M2 (Breakdown)
Status: Open

Now process this:
Comment: "{comment}"
→
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response['choices'][0]['message']['content']

# 🖥️ Gradio demo
demo = gr.Interface(
    fn=generate_work_order,
    inputs="text",
    outputs="text",
    title="GenAI SAP Work Order Generator",
    description="Paste any maintenance comment or inspection log to convert it into a structured work order."
)

demo.launch()